In [7]:
# At the top of your notebook
%load_ext autoreload
%autoreload 2
    
import os
import cv2
import torch
import numpy as np
import supervision as sv
from PIL import Image
from sam2.build_sam import build_sam2_video_predictor, build_sam2
from sam2.sam2_image_predictor import SAM2ImagePredictor
from transformers import AutoProcessor, AutoModelForZeroShotObjectDetection 
from utils.track_utils import sample_points_from_masks
from utils.video_utils import create_video_from_images
from utils.common_utils import CommonUtils
from utils.mask_dictionary_model import MaskDictionaryModel, ObjectInfo
import json
import copy

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
!nvidia-smi

Wed Jun 11 19:42:19 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA H100 80GB HBM3          On  | 00000000:53:00.0 Off |                    0 |
| N/A   23C    P0              67W / 700W |      3MiB / 81559MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [9]:
"""
Step 1: Environment settings and model initialization
"""
# use bfloat16 for the entire notebook
torch.autocast(device_type="cuda", dtype=torch.bfloat16).__enter__()

if torch.cuda.get_device_properties(0).major >= 8:
    # turn on tfloat32 for Ampere GPUs (https://pytorch.org/docs/stable/notes/cuda.html#tensorfloat-32-tf32-on-ampere-devices)
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True

# init sam image predictor and video predictor model
sam2_checkpoint = "./checkpoints/sam2.1_hiera_large.pt"
model_cfg = "configs/sam2.1/sam2.1_hiera_l.yaml"
device = "cuda" if torch.cuda.is_available() else "cpu"
print("device", device)

device cuda


In [10]:
device = "cuda:0"

In [11]:
video_predictor = build_sam2_video_predictor(model_cfg, sam2_checkpoint)
sam2_image_model = build_sam2(model_cfg, sam2_checkpoint, device=device)
image_predictor = SAM2ImagePredictor(sam2_image_model)

# init grounding dino model from huggingface
model_id = "IDEA-Research/grounding-dino-tiny"
processor = AutoProcessor.from_pretrained(model_id)
grounding_model = AutoModelForZeroShotObjectDetection.from_pretrained(model_id).to(device)

In [14]:
import glob
clips = glob.glob("outputs/frames_to_run/*")
clips

['outputs/frames_to_run/232f98b4-9293-46ea-95de-4fb4ae230648-part2-clip_00000003-0',
 'outputs/frames_to_run/9a15e066-f54e-4fe4-98cb-58f7d0f655c7-clip_00000121-4',
 'outputs/frames_to_run/Yca14HAvris-clip_00000079-2',
 'outputs/frames_to_run/677951670-clip_00000010-0',
 'outputs/frames_to_run/e5957a13-8fdd-4696-b8a5-11df9bdbb98d-part1-clip_00000039-0',
 'outputs/frames_to_run/6b887cf3-e579-4543-802d-708b7d598207-part3-clip_00000045-0',
 'outputs/frames_to_run/bLUOFZTDxXI-clip_00000010-2',
 'outputs/frames_to_run/xNVfw6I7cCA_part_000-clip_00000025-0',
 'outputs/frames_to_run/278950132-clip_00000001-0',
 'outputs/frames_to_run/442422784-clip_00000007-0',
 'outputs/frames_to_run/d7e18703-e8cd-4366-9821-e50e0c499176-part3-clip_00000037-0_anime',
 'outputs/frames_to_run/_fhf8qnFihk_part_000-clip_00000006-3',
 'outputs/frames_to_run/320150f9-253c-4d1a-b075-44e6eb264ec1-part6-clip_00000027-1',
 'outputs/frames_to_run/fOwKMumK46E-clip_00000006-2',
 'outputs/frames_to_run/cFKDTfrIpGQ_part_000-c

In [15]:
for video_dir in clips[:5]:
    # setup the input image and text prompt for SAM 2 and Grounding DINO
    # VERY important: text queries need to be lowercased + end with a dot
    #text = "car."
    #text = "person. motorcycle."
    #vid_id = "car
    text = "octopus. person. motorcycle. noodles. tree."
    
    #vid_id = "0alex_anime_3d_action_ufotable"
    #vid_id = "xNVfw6I7cCA_part_000-clip_00000025-0"
    vid_id = video_dir.split("/")[-1] #"tHEcy2tC6ug_part_000-clip_00000015-0"
    print(f"doing {vid_id}")
    
    import shutil
    
    # `video_dir` a directory of JPEG frames with filenames like `<frame_index>.jpg`  
    # video_dir = f"notebooks/videos/{vid_id}"
    
    # 'output_dir' is the directory to save the annotated frames
    output_dir = "./outputs"
    
    # 'output_video_path' is the path to save the final video
    output_video_path = f"./outputs/output_vids/{vid_id}.mp4"
    
    # create the output directory
    CommonUtils.creat_dirs(output_dir)
    mask_data_dir = os.path.join(output_dir, "mask_data")
    json_data_dir = os.path.join(output_dir, "json_data")
    result_dir = os.path.join(output_dir, "result")
    
    for d in [mask_data_dir, json_data_dir, result_dir]: # keep fresh for each vid
        if os.path.exists(d): shutil.rmtree(d)
            
    CommonUtils.creat_dirs(mask_data_dir)
    CommonUtils.creat_dirs(json_data_dir)
    # scan all the JPEG frame names in this directory
    frame_names = [
        p for p in os.listdir(video_dir)
        if os.path.splitext(p)[-1] in [".jpg", ".jpeg", ".JPG", ".JPEG", ".png", ".PNG"]
    ]
    frame_names.sort(key=lambda p: int(os.path.splitext(p)[0]))
    
    # init video predictor state
    inference_state = video_predictor.init_state(video_path=video_dir, offload_video_to_cpu=True, async_loading_frames=True)
    step = 20 # the step to sample frames for Grounding DINO predictor
    
    sam2_masks = MaskDictionaryModel()
    PROMPT_TYPE_FOR_VIDEO = "mask" # box, mask or point
    objects_count = 0
    
    """
    Step 2: Prompt Grounding DINO and SAM image predictor to get the box and mask for all frames
    """
    print("Total frames:", len(frame_names))
    
    for start_frame_idx in range(0, len(frame_names), step):
    # prompt grounding dino to get the box coordinates on specific frame
        print("start_frame_idx", start_frame_idx)
        # continue
        img_path = os.path.join(video_dir, frame_names[start_frame_idx])
        image = Image.open(img_path)
        image_base_name = frame_names[start_frame_idx].split(".")[0]
        mask_dict = MaskDictionaryModel(promote_type = PROMPT_TYPE_FOR_VIDEO, mask_name = f"mask_{image_base_name}.npy")
    
        # run Grounding DINO on the image
        inputs = processor(images=image, text=text, return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = grounding_model(**inputs)
    
        results = processor.post_process_grounded_object_detection(
            outputs,
            inputs.input_ids,
            box_threshold=0.25,
            text_threshold=0.25,
            target_sizes=[image.size[::-1]]
        )
    
        # prompt SAM image predictor to get the mask for the object
        image_predictor.set_image(np.array(image.convert("RGB")))
    
        # process the detection results
        input_boxes = results[0]["boxes"] # .cpu().numpy()
        # print("results[0]",results[0])
        OBJECTS = results[0]["labels"]
        if input_boxes.shape[0] != 0:
            # prompt SAM 2 image predictor to get the mask for the object
            masks, scores, logits = image_predictor.predict(
                point_coords=None,
                point_labels=None,
                box=input_boxes,
                multimask_output=False,
            )
            # convert the mask shape to (n, H, W)
            if masks.ndim == 2:
                masks = masks[None]
                scores = scores[None]
                logits = logits[None]
            elif masks.ndim == 4:
                masks = masks.squeeze(1)
    
            """
            Step 3: Register each object's positive points to video predictor
            """
    
            # If you are using point prompts, we uniformly sample positive points based on the mask
            if mask_dict.promote_type == "mask":
                mask_dict.add_new_frame_annotation(mask_list=torch.tensor(masks).to(device), box_list=torch.tensor(input_boxes), label_list=OBJECTS)
            else:
                raise NotImplementedError("SAM 2 video predictor only support mask prompts")
    
    
            """
            Step 4: Propagate the video predictor to get the segmentation results for each frame
            """
            objects_count = mask_dict.update_masks(tracking_annotation_dict=sam2_masks, iou_threshold=0.8, objects_count=objects_count)
            print("objects_count", objects_count)
        else:
            print("No object detected in the frame, skip merge the frame merge {}".format(frame_names[start_frame_idx]))
            mask_dict = sam2_masks
    
        
        if len(mask_dict.labels) == 0:
            mask_dict.save_empty_mask_and_json(mask_data_dir, json_data_dir, image_name_list = frame_names[start_frame_idx:start_frame_idx+step])
            print("No object detected in the frame, skip the frame {}".format(start_frame_idx))
            continue
        else: 
            video_predictor.reset_state(inference_state)
    
            for object_id, object_info in mask_dict.labels.items():
                frame_idx, out_obj_ids, out_mask_logits = video_predictor.add_new_mask(
                        inference_state,
                        start_frame_idx,
                        object_id,
                        object_info.mask,
                    )
            
            video_segments = {}  # output the following {step} frames tracking masks
            for out_frame_idx, out_obj_ids, out_mask_logits in video_predictor.propagate_in_video(inference_state, max_frame_num_to_track=step, start_frame_idx=start_frame_idx):
                frame_masks = MaskDictionaryModel()
                
                for i, out_obj_id in enumerate(out_obj_ids):
                    out_mask = (out_mask_logits[i] > 0.0) # .cpu().numpy()
                    object_info = ObjectInfo(instance_id = out_obj_id, mask = out_mask[0], class_name = mask_dict.get_target_class_name(out_obj_id))
                    object_info.update_box()
                    frame_masks.labels[out_obj_id] = object_info
                    image_base_name = frame_names[out_frame_idx].split(".")[0]
                    frame_masks.mask_name = f"mask_{image_base_name}.npy"
                    frame_masks.mask_height = out_mask.shape[-2]
                    frame_masks.mask_width = out_mask.shape[-1]
    
                video_segments[out_frame_idx] = frame_masks
                sam2_masks = copy.deepcopy(frame_masks)
    
            print("video_segments:", len(video_segments))
        """
        Step 5: save the tracking masks and json files
        """
        for frame_idx, frame_masks_info in video_segments.items():
            mask = frame_masks_info.labels
            mask_img = torch.zeros(frame_masks_info.mask_height, frame_masks_info.mask_width)
            for obj_id, obj_info in mask.items():
                mask_img[obj_info.mask == True] = obj_id
    
            mask_img = mask_img.numpy().astype(np.uint16)
            np.save(os.path.join(mask_data_dir, frame_masks_info.mask_name), mask_img)
    
            json_data = frame_masks_info.to_dict()
            json_data_path = os.path.join(json_data_dir, frame_masks_info.mask_name.replace(".npy", ".json"))
            with open(json_data_path, "w") as f:
                json.dump(json_data, f)

        
    """
    Step 6: Draw the results and save the video
    """
    CommonUtils.draw_masks_and_box_with_supervision(video_dir, mask_data_dir, json_data_dir, result_dir)
    
    create_video_from_images(result_dir, output_video_path, frame_rate=15)


doing 232f98b4-9293-46ea-95de-4fb4ae230648-part2-clip_00000003-0
Path './outputs' already exists.
Path './outputs/mask_data' did not exist and has been created.
Path './outputs/json_data' did not exist and has been created.


frame loading (JPEG):   8%|█▋                    | 5/65 [00:00<00:01, 43.20it/s]

Total frames: 65
start_frame_idx 0
objects_count 3


frame loading (JPEG):  80%|████████████████▊    | 52/65 [00:01<00:00, 38.97it/s]

video_segments: 21
start_frame_idx 20


...

propagate in video: 100%|█████████████████████████| 2/2 [00:00<00:00, 48.99it/s]

video_segments: 2
Path './outputs/result' did not exist and has been created.
raw image name list ['00000.jpg', '00001.jpg', '00002.jpg', '00003.jpg', '00004.jpg', '00005.jpg', '00006.jpg', '00007.jpg', '00008.jpg', '00009.jpg', '00010.jpg', '00011.jpg', '00012.jpg', '00013.jpg', '00014.jpg', '00015.jpg', '00016.jpg', '00017.jpg', '00018.jpg', '00019.jpg', '00020.jpg', '00021.jpg', '00022.jpg', '00023.jpg', '00024.jpg', '00025.jpg', '00026.jpg', '00027.jpg', '00028.jpg', '00029.jpg', '00030.jpg', '00031.jpg', '00032.jpg', '00033.jpg', '00034.jpg', '00035.jpg', '00036.jpg', '00037.jpg', '00038.jpg', '00039.jpg', '00040.jpg', '00041.jpg', '00042.jpg', '00043.jpg', '00044.jpg', '00045.jpg', '00046.jpg', '00047.jpg', '00048.jpg', '00049.jpg', '00050.jpg', '00051.jpg', '00052.jpg', '00053.jpg', '00054.jpg', '00055.jpg', '00056.jpg', '00057.jpg', '00058.jpg', '00059.jpg', '00060.jpg', '00061.jpg', '00062.jpg', '00063.jpg', '00064.jpg', '00065.jpg', '00066.jpg', '00067.jpg', '00068.jpg', '000

Annotated image saved as ./outputs/result/00000.jpg
Annotated image saved as ./outputs/result/00001.jpg
Annotated image saved as ./outputs/result/00002.jpg
Annotated image saved as ./outputs/result/00003.jpg
Annotated image saved as ./outputs/result/00004.jpg
Annotated image saved as ./outputs/result/00005.jpg
Annotated image saved as ./outputs/result/00006.jpg
Annotated image saved as ./outputs/result/00007.jpg
Annotated image saved as ./outputs/result/00008.jpg
Annotated image saved as ./outputs/result/00009.jpg
Annotated image saved as ./outputs/result/00010.jpg
Annotated image saved as ./outputs/result/00011.jpg
Annotated image saved as ./outputs/result/00012.jpg
Annotated image saved as ./outputs/result/00013.jpg
Annotated image saved as ./outputs/result/00014.jpg
Annotated image saved as ./outputs/result/00015.jpg
Annotated image saved as ./outputs/result/00016.jpg
Annotated image saved as ./outputs/result/00017.jpg
Annotated image saved as ./outputs/result/00018.jpg
Annotated im

100%|████████████████████████████████████████| 142/142 [00:01<00:00, 121.41it/s]

Video saved at ./outputs/output_vids/e5957a13-8fdd-4696-b8a5-11df9bdbb98d-part1-clip_00000039-0.mp4


In [9]:
video_dir

'notebooks/videos/tHEcy2tC6ug_part_000-clip_00000015-0'

Path './outputs/result' did not exist and has been created.
raw image name list ['.ipynb_checkpoints', '00000.jpg', '00001.jpg', '00002.jpg', '00003.jpg', '00004.jpg', '00005.jpg', '00006.jpg', '00007.jpg', '00008.jpg', '00009.jpg', '00010.jpg', '00011.jpg', '00012.jpg', '00013.jpg', '00014.jpg', '00015.jpg', '00016.jpg', '00017.jpg', '00018.jpg', '00019.jpg', '00020.jpg', '00021.jpg', '00022.jpg', '00023.jpg', '00024.jpg', '00025.jpg', '00026.jpg', '00027.jpg', '00028.jpg', '00029.jpg', '00030.jpg', '00031.jpg', '00032.jpg', '00033.jpg', '00034.jpg', '00035.jpg', '00036.jpg', '00037.jpg', '00038.jpg', '00039.jpg', '00040.jpg', '00041.jpg', '00042.jpg', '00043.jpg', '00044.jpg', '00045.jpg', '00046.jpg', '00047.jpg', '00048.jpg', '00049.jpg', '00050.jpg', '00051.jpg', '00052.jpg', '00053.jpg', '00054.jpg', '00055.jpg', '00056.jpg', '00057.jpg', '00058.jpg', '00059.jpg', '00060.jpg', '00061.jpg', '00062.jpg', '00063.jpg', '00064.jpg', '00065.jpg', '00066.jpg', '00067.jpg', '00068.jpg', 

100%|███████████████████████████████████████████████████████████████| 258/258 [00:03<00:00, 69.24it/s]

Video saved at ./outputs/tHEcy2tC6ug_part_000-clip_00000015-0.mp4
